In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
import time
import argparse

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
from sql_init import Review, Base, Category, Product
from scrape import check_category, check_product, scrape_category, scrape_product,scrape_reviews

In [2]:
def check_category(e, session):
    categories = session.query(Category).filter_by(category_name=e.text).all()
    if len(categories) == 0:
        new_category = Category(category_name=e.text,completed=False)
        session.add(new_category)
        session.commit()
        return False
    elif len(categories) >= 1:
        return categories[0].completed
    
def check_product(product, session):
    product_id = re.search('/dp/[\w]+/',product.find_element_by_xpath('..').get_attribute('href')).group(0)[4:-1]
    products = session.query(Product).filter_by(product_ASIN=product_id).all()
    if len(products) == 0:
        new_product = Product(product_ASIN=product_id,completed=False)
        session.add(new_product)
        session.commit()
        return False
    elif len(products) >= 1:
        return products[0].completed

def scrape_review(review, session, product_id):
    star_rating = review.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
    e = review.find_element_by_xpath('../../../..')
    if len(session.query(Review).filter_by(review_id=e.get_attribute('id')).all()) == 0:
        new_review = Review(review_id=e.get_attribute('id'),
                            star_rating=int(star_rating),
                            review_title=e.find_element_by_class_name('review-title').text,
                            review_date=e.find_element_by_class_name('review-date').text[3:],
                            text=e.find_element_by_class_name('review-text').text,
                            product_ASIN = product_ASIN)
        session.add(new_review)
        session.commit()
        return
    else:
        pass

In [3]:
def scrape_reviews(link, driver, session, product_id):
    link.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
    driver.switch_to_window(driver.window_handles[3])
    while True:
        time.sleep(1)
        reviews = driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]')
        try:
            for review in reviews:
                star_rating = review.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
                e = review.find_element_by_xpath('../../../..')
                if len(session.query(Review).filter_by(review_id=e.get_attribute('id')).all()) == 0:
                    new_review = Review(review_id=e.get_attribute('id'),
                                        star_rating=int(star_rating),
                                        review_title=e.find_element_by_class_name('review-title').text,
                                        review_date=e.find_element_by_class_name('review-date').text[3:],
                                        text=e.find_element_by_class_name('review-text').text,
                                        product_ASIN = product_id)
                    session.add(new_review)
                    session.commit()
                else:
                    pass
            time.sleep(1)
            driver.execute_script("window.scrollTo(0, 10000)")
            next_ = driver.find_element_by_class_name('a-last')
            next_link = next_.find_element_by_tag_name('a')
            next_link.click()
        except NoSuchElementException:
        #     cat = session.query(Category).filter_by(category_name=e.text).first()
        #     cat.completed = True
        #     session.commit()
            driver.close()
            driver.switch_to_window(driver.window_handles[2])
            break


                #TODO What if there are no reviews

In [4]:
def scrape_product(product, driver, session):
    if not check_product(product, session):
        product_ASID = re.search('/dp/[\w]+/',product.find_element_by_xpath('..').get_attribute('href')).group(0)[4:-1]
        product.find_element_by_xpath('..').send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
        driver.switch_to_window(driver.window_handles[2])
        
        links = driver.find_elements_by_tag_name('a')
        for link in links:
            if re.search('See all [0-9,.]* customer reviews', link.text):
                scrape_reviews(link, driver, session, product_ASID)
                break

        new_product = session.query(Product).filter_by(product_ASIN=product_ASID).first()
        new_product.completed = True
        session.commit()
        driver.close()
        driver.switch_to_window(driver.window_handles[1])
        
        

In [5]:
def scrape_category(category, driver, session):
    category_text = category.text
    
    if not check_category(category, session):
        print(category.find_element_by_xpath('..').tag_name)
        category.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
        driver.switch_to_window(driver.window_handles[1])
    while True:
        try:
            for i, product in enumerate(driver.find_elements_by_class_name("s-access-title")):
                driver.find_elements_by_class_name("s-access-title")[i]
                scrape_product(product, driver, session)
            time.sleep(2) 
            driver.execute_script("window.scrollTo(0, 10000)")
            next_products = driver.find_element_by_id('pagnNextLink')
            next_products.click()

        except NoSuchElementException:
            #     cat = session.query(Category).filter_by(category_name=e.text).first()
            #     cat.completed = True
            #     session.commit()
            driver.close()
            driver.switch_to_window(driver.window_handles[0])
            break

        cat = session.query(Category).filter_by(category_name=category_text).first()
        cat.completed = True
        session.commit()  

In [6]:
# Open sqlite connection/session
engine = create_engine('sqlite:///reviews.db')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()


In [51]:

# Open a new browser window
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.implicitly_wait(1.5)


In [52]:

# Prompt the user for 
# url = raw_input("What amazon reviews would you like to scrape?:")
# if url == '':
url = 'https://www.amazon.com/s/gp/search/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aadidas%2Cp_89%3Aadidas&keywords=adidas&ie=UTF8&qid=1511382355&rnid=2528832011'
driver.get(url)

main_window = driver.current_window_handle


In [53]:
# Expand all of the "see more" elements
for i in driver.find_elements_by_class_name("a-expander-prompt"):
    i.find_element_by_xpath('..').click()



In [54]:
nav = driver.find_element_by_id("leftNavContainer")

In [55]:
categories = nav.find_elements_by_class_name('a-spacing-top-mini')

In [56]:
category = categories[0]
# for category in categories:

In [ ]:
scrape_category(category, driver, session)

In [45]:
if not check_category(category, session):
# Open new window for category
    category.find_element_by_xpath('..').send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)

In [46]:
driver.switch_to_window(driver.window_handles[1])

In [47]:
product = driver.find_elements_by_class_name("s-access-title")[0]
# for product in driver.find_elements_by_class_name("s-access-title"):

In [ ]:
product_ASIN = re.search('/dp/[\w]+/',product.find_element_by_xpath('..').get_attribute('href')).group(0)[4:-1]

In [ ]:
if not check_product(product,session):
    pass

In [ ]:
scrape_product(product, driver, session)

In [48]:
product.find_element_by_xpath('..').send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)

In [49]:
driver.switch_to_window(driver.window_handles[2])

In [ ]:
links = driver.find_elements_by_tag_name('a')

In [ ]:
for link in links:
    if re.search('See all [0-9,.]* customer reviews', link.text):
        link.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
        driver.switch_to_window(driver.window_handles[3])
        #TODO What if there are no reviews
        break   

In [ ]:
for link in links:
    if re.search('See all [0-9,.]* customer reviews', link.text):
        scrape_reviews(link, driver, session)
        break  


In [ ]:
for review in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
    scrape_review(review, session)

In [ ]:
driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]')

In [ ]:
while True:
    time.sleep(1)
    reviews = driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]')
    try:
        for review in reviews:
            star_rating = review.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
            e = review.find_element_by_xpath('../../../..')
            if len(session.query(Review).filter_by(review_id=e.id).all()) > 0:
                new_review = Review(review_id=e.get_attribute('id'),
                                    star_rating=int(star_rating),
                                    review_title=e.find_element_by_class_name('review-title').text,
                                    review_date=e.find_element_by_class_name('review-date').text[3:],
                                    text=e.find_element_by_class_name('review-text').text)
                session.add(new_review)
                session.commit()
            else:
                pass
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 10000)")
        next_ = driver.find_element_by_class_name('a-last')
        next_link = next_.find_element_by_tag_name('a')
        next_link.click()
    except NoSuchElementException:
    #     cat = session.query(Category).filter_by(category_name=e.text).first()
    #     cat.completed = True
    #     session.commit()
        driver.close()
        driver.switch_to_window(driver.window_handles[2])
        driver.close()
        driver.switch_to_window(driver.window_handles[1])
        break

In [ ]:
# driver.execute_script("window.scrollTo(0, 10000)")
next_products = driver.find_element_by_id('pagnNextString')
# next_products.find_element_by_xpath('..').click()
next_products.text

In [ ]:
driver.execute_script("window.scrollTo(0, 10000)")
next_products = driver.find_element_by_id('pagnNextLink')
next_products.click()

In [ ]:
while True:
    try:
        time.sleep(2) 
        driver.execute_script("window.scrollTo(0, 10000)")
        next_products = driver.find_element_by_id('pagnNextLink')
        next_products.click()

    except NoSuchElementException:
        #     cat = session.query(Category).filter_by(category_name=e.text).first()
        #     cat.completed = True
        #     session.commit()
        driver.close()
        driver.switch_to_window(driver.window_handles[0])
        break

In [ ]:
driver.find_element_by_id('pagnNextString').find_element_by_xpath('..').click()

In [ ]:
# try:
# time.sleep(1)
e = driver.find_element_by_class_name('a-last')
f = e.find_elements_by_tag_name('a')
# except NoSuchElementException:
#     cat = session.query(Category).filter_by(category_name=e.text).first()
#     cat.completed = True
#     session.commit()
#     driver.close()
#     driver.switch_to_window(driver.window_handles[1])
#     break

In [ ]:
f

In [ ]:
# Iterate through the categories
categories = []
for category in driver.find_elements_by_tag_name('h4')#.find_elements_by_class_name("a-text-bold"):
    # Check if Category has been scraped
#     if category.text == "Get It by Tomorrow":
#         break
    print(category.text)
#     # If not complete -> scrape
#     if not check_category(category, session):
#         # Open new window for category
#         category.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
#         # Switch to new window 
#         driver.switch_to_window(driver.window_handles[1])
#         for product in driver.find_elements_by_class_name("s-access-title"):
#             # time.sleep(1)
#             product.click()
#             # driver.switch_to_window(driver.window_handles[2])
#             a_s = driver.find_elements_by_tag_name('a')
#             for e in a_s:
#                 time.sleep(1)
#                 if re.search('See all [0-9,.]* customer reviews', e.text):
#                     e.send_keys(Keys.CONTROL + Keys.SHIFT + Keys.RETURN)
#                     driver.switch_to_window(driver.window_handles[2])
#                     break
#             while True:
#                 time.sleep(1)
#                 for review in driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]'):
#                     scrape_review(review, session)

#                 try:
#                     time.sleep(1)
#                     e = driver.find_element_by_class_name('a-last')
#                     e.find_element_by_tag_name('a').click()
#                 except NoSuchElementException:
#                     cat = session.query(Category).filter_by(category_name=e.text).first()
#                     cat.completed = True
#                     session.commit()
#                     driver.close()
#                     driver.switch_to_window(driver.window_handles[1])
#                     break
#             driver.execute_script("window.history.go(-1)")



In [ ]:
review = driver.find_elements_by_xpath('*//i[@data-hook="review-star-rating"]')[0]
star_rating = review.find_element_by_class_name('a-icon-alt').get_attribute('innerHTML')[0]
# e = review.find_element_by_xpath('../../../..')
# if len(session.query(Review).filter_by(review_id=e.id).all()) > 0:
#     new_review = Review(review_id=e.get_attribute('id'),
#                         star_rating=int(star_rating),
#                         review_title=e.find_element_by_class_name('review-title').text,
#                         review_date=e.find_element_by_class_name('review-date').text[3:],
#                         text=e.find_element_by_class_name('review-text').text,
#                         product_ASIN = product_id)
#     session.add(new_review)
#     session.commit()

In [ ]:
e = review.find_element_by_xpath('../../../..')

In [ ]:
session.rollback()

In [ ]:
if len(session.query(Review).filter_by(review_id=e.get_attribute('id')).all()) == 0:
    new_review = Review(review_id=e.get_attribute('id'),
                        star_rating=int(star_rating),
                        review_title=e.find_element_by_class_name('review-title').text,
                        review_date=e.find_element_by_class_name('review-date').text[3:],
                        text=e.find_element_by_class_name('review-text').text,
                        product_ASIN = product_ASIN)
    session.add(new_review)
    session.commit()
    print('blah')

In [ ]:
len(session.query(Review).filter_by(review_id=e.get_attribute('id')).all())

In [72]:
len(session.query(Review).all())

392

In [20]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException, NoSuchElementException
from bs4 import BeautifulSoup
import requests
import re
from collections import defaultdict
import time
import argparse

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
 
from sql_init import Review, Base, Category, Product
from scrape import check_category, check_product, scrape_category, scrape_product,scrape_reviews
# Open sqlite connection/session
engine = create_engine('sqlite:///reviews.db')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

# Open a new browser window
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.implicitly_wait(1.5)

# Prompt the user for
# url = raw_input("What amazon reviews would you like to scrape?:")
# if url == '':
url = 'https://www.amazon.com/s/gp/search/ref=sr_nr_p_89_0?fst=as%3Aoff&rh=i%3Aaps%2Ck%3Aadidas%2Cp_89%3Aadidas&keywords=adidas&ie=UTF8&qid=1511382355&rnid=2528832011'
driver.get(url)

main_window = driver.current_window_handle

# Expand all of the "see more" elements
for i in driver.find_elements_by_class_name("a-expander-prompt"):
    i.find_element_by_xpath('..').click()

# Iterate through the categories
categories = []
for i, category in enumerate(driver.find_elements_by_class_name("s-ref-text-link")):
    category_ = driver.find_elements_by_class_name("s-ref-text-link")[i]
    scrape_category(category_, driver, session)

IndexError: list index out of range

In [ ]:
for i, category in enumerate(driver.find_elements_by_class_name("s-ref-text-link")):
    category_ = driver.find_elements_by_class_name("s-ref-text-link")[i]
    print(category_.text)

In [33]:
check_category(category, session)

True

In [57]:
b = session.query(Category).filter_by(category_name="Boys' Fashion").first()

In [58]:
b.completed = False

In [50]:
driver.execute_script("window.scrollTo(0, 10000)")
next_products = driver.find_element_by_id('pagnNextLink')
next_products.click()

In [59]:
session.commit()

In [64]:
[x.category_name for x in session.query(Category).all()]

['Musical Instruments',
 'Pet Supplies',
 'Garden & Outdoor',
 'CDs & Vinyl',
 'Video Games',
 'Arts, Crafts & Sewing']

In [ ]:
Review.

In [32]:
engine = create_engine('sqlite:///reviews.db')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()
len(session.query(Product).all())

1498

In [31]:
len(set([x.review_id for x in session.query(Review).all()]))

62994

In [85]:
[x.star_rating for x in session.query(Review).all()[-5:]]

['5', '5', '5', '5', '5']

In [33]:
len(session.query(Product).all())

1498

In [124]:
# [x.category_name for x in session.query(Category).all()]
for x in session.query(Category).all():
    x.completed = False 
session.commit()

In [125]:
[x.completed for x in session.query(Category).all()]

[False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [14]:
p = session.query(Product).all()

In [20]:
p[1].product_ASIN

'B071LGH5BJ'